<a href="https://colab.research.google.com/github/Faizanealiqazi/WeatherDataScrapper/blob/main/WeatherDataScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta

In [2]:
dates = []
start_date = datetime.strptime("01-1942", "%m-%Y")
end_date = datetime.strptime("12-2023", "%m-%Y")
current_date = start_date

while current_date <= end_date:
    dates.append(current_date.strftime("%m-%Y"))
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1)

In [3]:
#for second run for extracting data for new dates which were not available before
dates = []
start_date = datetime.strptime("11-2023", "%m-%Y")
end_date = datetime.strptime("12-2023", "%m-%Y")
current_date = start_date

while current_date <= end_date:
    dates.append(current_date.strftime("%m-%Y"))
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1)

In [4]:
dates

['11-2023', '12-2023']

In [5]:
for date in dates:
    current_file = date+'.csv'
    print(f"Extracting for date: {date}")
    #url =''
    url = 'https://en.tutiempo.net/climate/' + date + '/ws-417800.html'
    print(f"Hitting url : {url}")

    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        style_map = soup.find_all('style')
        if len(style_map) > 1:
            style_content = str(style_map[1])
        else:
            print(f"No style map found for date: {date}")
            continue

        css_classes = re.findall(r'\.numspan span\.(nt\w+)::after\{content:"(.*?)";', style_content)
        replacement = {f'<span class="{cls}"></span>': f'<span class="{cls}">{content}</span>' for cls, content in css_classes}

        all_tables = soup.find_all('table')
        if len(all_tables) > 3:
            table = all_tables[3]
        else:
            print(f"No table found for date: {date}")
            continue

        html_str = str(table)

        for old, new in replacement.items():
            html_str = html_str.replace(old, new)

        updated_table = BeautifulSoup(html_str, 'html.parser')

        if updated_table:
            headers = [th.get_text(strip=True) for th in updated_table.find('tr').find_all('th')] if updated_table.find('tr') else []
            if not headers:
                print(f"No headers found for date: {date}")
                continue

            rows = []
            for row in updated_table.find_all('tr')[1:]:
                if row.th and row.th.has_attr('colspan'):
                    continue  # Skip this row
                cols = row.find_all('td')
                row_data = [td.get_text(strip=True) for td in cols]
                if row_data:
                    rows.append(row_data)

            df = pd.DataFrame(rows, columns=headers)
            df.to_csv(current_file, index=False)
        else:
            print(f"Could not update table for date: {date}")

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error for date {date}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Request Exception for date {date}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred for date {date}: {e}")


Extracting for date: 11-2023
Hitting url : https://en.tutiempo.net/climate/11-2023/ws-417800.html
Extracting for date: 12-2023
Hitting url : https://en.tutiempo.net/climate/12-2023/ws-417800.html


In [6]:
import os
df = []
merged_df = []
for date in dates:
    file_name = f"{date}.csv"
    print(f"Processing file: {file_name}")
    if not os.path.isfile(file_name):
        print(f"File {file_name} does not exist.")
        continue

    try:
        current_df = pd.read_csv(file_name)
        current_df = current_df[:-1]
        current_df['date'] = date
        merged_df.append(current_df)

    except pd.errors.EmptyDataError:
        print(f"No data in file {file_name}, skipping.")
    except pd.errors.ParserError:
        print(f"Parsing error in file {file_name}, skipping.")
    except Exception as e:
        print(f"An unexpected error occurred while processing {file_name}: {e}")

df = pd.concat(merged_df, ignore_index=True)

Processing file: 11-2023.csv
Processing file: 12-2023.csv


In [7]:
df['Day'] = df['Day'].astype(int).astype(str)
df['date'] = df['Day'] + '-' + df['date']
print(df)

   Day     T    TM    Tm     SLP     H   PP   VV    V    VM VG   RA   SN   TS  \
0    1  30.6  37.9  25.0  1013.9  39.0  0.0  4.0  4.6  11.1  -  NaN  NaN  NaN   
1    2  30.3  38.2  23.5  1013.2  37.0  0.0  4.0  5.6  14.8  -  NaN  NaN  NaN   
2    3  29.8  36.8  24.4  1010.9  51.0  0.0  4.0  8.7  18.3  -  NaN  NaN  NaN   
3    4  29.8  34.1  26.0  1011.3  68.0  0.0  4.0  5.7  14.8  -  NaN  NaN  NaN   
4    5  28.8  36.1  23.5  1013.1  59.0  0.0  3.7  4.6  14.8  -  NaN  NaN  NaN   
..  ..   ...   ...   ...     ...   ...  ...  ...  ...   ... ..  ...  ...  ...   
56  27     -     -     -       -     -    -    -    -     -  -    -    -    -   
57  28     -     -     -       -     -    -    -    -     -  -    -    -    -   
58  29     -     -     -       -     -    -    -    -     -  -    -    -    -   
59  30     -     -     -       -     -    -    -    -     -  -    -    -    -   
60  31     -     -     -       -     -    -    -    -     -  -    -    -    -   

     FG        date  
0   N

In [8]:
df.tail(30)

,Day,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG,date
31,2,23.2,29.3,16.5,1016.3,44,0,3.4,12.4,18.3,-,NaN,NaN,NaN,NaN,2-12-2023
32,3,23.3,30.2,17.5,1016.6,41,0,3.2,9.3,18.3,-,NaN,NaN,NaN,NaN,3-12-2023
33,4,22.6,30.5,15.1,1016.2,43,0,3.7,7,11.1,-,NaN,NaN,NaN,NaN,4-12-2023
34,5,22.4,30.2,15,1015.5,43,0,4,5.6,14.8,-,NaN,NaN,NaN,NaN,5-12-2023
35,6,-,-,-,-,-,-,-,-,-,-,-,-,-,-,6-12-2023
36,7,-,-,-,-,-,-,-,-,-,-,-,-,-,-,7-12-2023
37,8,-,-,-,-,-,-,-,-,-,-,-,-,-,-,8-12-2023
38,9,-,-,-,-,-,-,-,-,-,-,-,-,-,-,9-12-2023
39,10,-,-,-,-,-,-,-,-,-,-,-,-,-,-,10-12-2023
40,11,-,-,-,-,-,-,-,-,-,-,-,-,-,-,11-12-2023


In [9]:
rename_columns = {
    'T' : 'Average Temperature (°C)',
    'TM': 'Maximum temperature (°C)',
    'Tm': 'Minimum temperature (°C)',
    'SLP':'Atmospheric pressure at sea level (hPa)',
    'H' : 'Average relative humidity (%)',
    'PP': 'Total rainfall (mm)',
    'VV': 'Average visibility (Km)',
    'V' : 'Average wind speed (Km/h)',
    'VM': 'Maximum sustained wind speed (Km/h)',
    'VG': 'Maximum speed of wind (Km/h)',
    'RA': 'Total days it rained',
    'SN': 'Total days that snowed',
    'TS': 'Total days with thunderstorm',
    'FG': 'Total days with fog',
}
df.rename(columns=rename_columns, inplace=True)

In [10]:
df.to_csv('WeatherData1942-2023.csv', index=False)
#df.to_csv('WeatherData11_2023-12_2023.csv', index=False) #Uncomment for second run

In [ ]:
!zip -r /content/WeatherData.zip /content/*.csv